#### Imports

In [6]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")


In [7]:
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()

fs = project.get_feature_store()

2025-01-07 18:43:50,591 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 18:43:50,594 INFO: Initializing external client
2025-01-07 18:43:50,594 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 18:43:52,937 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207502


#### Variables

In [8]:
country="Sweden"
city = "Stockholm"
street = ""
aqicn_url=""
latitude = "59.3294"
longitude = "18.0687"
today = datetime.date.today()

#### Connect to Hopsworks 

In [9]:
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    
project = hopsworks.login()
fs = project.get_feature_store()

2025-01-07 18:43:55,849 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 18:43:55,851 INFO: Initializing external client
2025-01-07 18:43:55,851 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 18:43:57,559 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207502


### Accident data


In [19]:
csv_file="../data/police_api_backfill.csv"
util.check_file_path(csv_file)

df_acc = pd.read_csv(csv_file, sep=',', skipinitialspace=True)
df_acc['date'] = pd.to_datetime(df_acc['date'])
df_acc['accidents'] = df_acc['accidents'].astype('float32')
df_acc['country'] = country
df_acc['city'] = city
df_acc['day_of_week'] = df_acc['date'].dt.dayofweek
df_acc.dropna(subset=['date'], inplace=True)
df_acc

File successfully found at the path: ../data/police_api_backfill.csv


,date,accidents,country,city,day_of_week
0,2024-08-01,10.0,Sweden,Stockholm,3
1,2024-08-02,16.0,Sweden,Stockholm,4
2,2024-08-03,9.0,Sweden,Stockholm,5
3,2024-08-04,8.0,Sweden,Stockholm,6
4,2024-08-05,15.0,Sweden,Stockholm,0
...,...,...,...,...,...
154,2025-01-02,21.0,Sweden,Stockholm,3
155,2025-01-03,25.0,Sweden,Stockholm,4
156,2025-01-04,14.0,Sweden,Stockholm,5
157,2025-01-05,15.0,Sweden,Stockholm,6


#### Create Accidents Feature Group

In [20]:
accidents_fg = fs.get_or_create_feature_group(
    name='accidents',
    description='Traffic accidents of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
)

#### Insert the DataFrame into the Feature Group

In [21]:
accidents_fg.insert(df_acc)

Uploading Dataframe: 100.00% |██████████| Rows 159/159 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('accidents_1_offline_fg_materialization', 'SPARK'), None)

#### Enter a description for each feature in the Feature Group

In [13]:
accidents_fg.update_feature_description("date", "Date of accidents")
accidents_fg.update_feature_description("country", "Country where the accidents was recorded")
accidents_fg.update_feature_description("city", "City where the the accidents was recorded")
accidents_fg.update_feature_description("accidents", "Number of traffic accidents reported to the police")
accidents_fg.update_feature_description("day_of_week", "Day of the week of the accident")

### Weather data

In [ ]:
earliest_aq_date = pd.Series.min(df_acc['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)
weather_df['day_of_week'] = weather_df['date'].dt.dayofweek
weather_df.dropna(subset=['date'], inplace=True)


weather_df.info()
weather_df.head()

Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
<class 'pandas.core.frame.DataFrame'>
Index: 158 entries, 0 to 157
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         158 non-null    datetime64[ns]
 1   temperature_2m_mean          158 non-null    float32       
 2   precipitation_sum            158 non-null    float32       
 3   wind_speed_10m_max           158 non-null    float32       
 4   wind_direction_10m_dominant  158 non-null    float32       
 5   city                         158 non-null    object        
 6   day_of_week                  158 non-null    int32         
dtypes: datetime64[ns](1), float32(4), int32(1), object(1)
memory usage: 6.8+ KB


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,day_of_week
0,2024-08-01,17.496252,0.0,14.764390,316.718292,Stockholm,3
1,2024-08-02,17.025417,0.0,11.885453,317.973785,Stockholm,4
2,2024-08-03,17.908751,0.5,10.009036,319.460907,Stockholm,5
3,2024-08-04,17.523336,8.5,7.208994,115.214478,Stockholm,6
4,2024-08-05,18.685835,0.4,8.654986,34.007500,Stockholm,0


#### Create Weather Feature Group

In [26]:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
) 

#### Insert the DataFrame into the Feature Group

In [27]:
weather_fg.insert(weather_df)

Uploading Dataframe: 100.00% |██████████| Rows 158/158 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('weather_1_offline_fg_materialization', 'SPARK'), None)

#### Enter a description for each feature in the Feature Group

In [17]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")
weather_fg.update_feature_description("day_of_week", "Day of the week of the weather measurement")